In [4]:
import numpy as np

In [99]:
class SeqModel:
    def __init__(self, arrOfLayers=[]):
        self.layers = arrOfLayers
    
    def add(self, layer):
        self.layers.append(layer)
    
    def summary(self):
        for layer in self.layers:
            print(layer)
    
    def inp(self, inp):
        self.layers[0].output = inp
    
    def forward_propagate(self):
        for i in range(1,len(self.layers)):
            self.layers[i].forward(self.layers[i-1].output)
        return self.layers[-1].output
        
    def back_propagate(self):
        pass

class Layer:
    def __init__(self):
        pass
    
    def forward(self, inp):
        pass
    
class InputLayer:
    def __init__(self, size=0, inp=[]):
        self.output = inp
    
    def __repr__(self):
        return ""

class OutputLayer:
    def __init__(self):
        self.output = []
    
    def forward(self, inp):
        self.output = inp
        return(inp)
    
    def __repr__(self):
        return ""

In [151]:
class ConvolutionalLayer:
    # size input dan size filter menggunakan format (jumlah_channel, tinggi, lebar)
    # size padding dan stride meunggunakan format (tinggi, lebar)
#     def __init__(self, size_input: tuple[int, int, int], padding: tuple[int, int], n_filter: int, size_filter: tuple[int, int, int], size_stride: tuple[int, int]):
#         if(size_input[0] != size_filter[0]):
    def __init__(self, size_input, padding, n_filter, size_filter, size_stride, name="convoboi"):
        if(size_input[0] != size_filter[0]):
            raise ValueError('Jumlah channel input dan filter tidak sesuai!!')
        self.size_input = size_input
        self.padding = padding
        self.n_filter = n_filter
        self.size_filter = size_filter
        self.shape_filter = (n_filter,) + size_filter
        self.size_stride = size_stride
        self.filters = np.zeros(self.shape_filter)
        self.bias = np.zeros(n_filter)
        self.detector = 'ReLU'
        self.name = name
        
    def __repr__(self):
        return f"\nConvolutional layer : {self.name}\nOutput_Shape : {self.n_filter}*{self.size_filter[1]}*{self.size_filter[2]}\nNParams : {(self.n_filter*self.size_filter[2]*self.size_filter[1]+1) * self.size_filter[0]}\n"

#     def set_detector(self, detector: str = 'ReLU'):
    def set_detector(self, detector='ReLU'):
        if(detector == 'ReLU' or detector == 'sigmoid'):
            self.detector = detector
        else:
            raise ValueError('Detector yang tersedia hanya ReLU dan sigmoid.')

#     def set_filter(self, filters: np.ndarray):
    def set_filter(self, filters):
        if filters.shape != self.shape_filter:
            raise ValueError('Shape filter tidak sesuai.')
        self.filters = filters

    def calculate_receptive_map_size(self):
        return (self.size_filter[0],int((self.size_input[1]-self.size_filter[1]+self.padding[0]+self.size_stride[0])/self.size_stride[0]), int((self.size_input[2]-self.size_filter[2]+self.padding[1]+self.size_stride[1])/self.size_stride[1]))

#     def conv(self, input: np.ndarray):
    def forward(self, input):
        if(input.shape != self.size_input):
            raise ValueError('Input tidak sesuai!!')
        shape_receptive_field = self.calculate_receptive_map_size()
        output = np.zeros((self.n_filter, shape_receptive_field[1], shape_receptive_field[2]))
        for filter in range(self.n_filter):
            for channel in range(shape_receptive_field[0]):
                # print(output)
                for t in range(shape_receptive_field[1]):
                    for l in range(shape_receptive_field[2]):
                        rec_start_t = max(0,t*self.size_stride[0]-self.padding[0])
                        rec_end_t = min(input.shape[1],t*self.size_stride[0]-self.padding[0]+self.size_filter[1])
                        rec_start_l = max(0,l*self.size_stride[1]-self.padding[1])
                        rec_end_l = min(input.shape[2],l*self.size_stride[1]-self.padding[1]+self.size_filter[2])
                        # print (filter, channel, t, l, rec_start_t, rec_end_t, rec_start_l, rec_end_l)
                        # print(input[channel,rec_start_t:rec_end_t,rec_start_l:rec_end_l])
                        # print(self.filters[filter, channel])
                        # print(np.sum(np.multiply(input[channel,rec_start_t:rec_end_t,rec_start_l:rec_end_l],self.filters[filter, channel])) + self.bias[filter])
                        output[filter, t, l] += np.sum(np.multiply(input[channel,rec_start_t:rec_end_t,rec_start_l:rec_end_l],self.filters[filter, channel])) + self.bias[filter]
        self.output = output
        return output

conv = ConvolutionalLayer((3,3,3),(0,0),2,(3,2,2),(1,1))
filter1 = [[[0,-1],
            [1,0]],

            [[5,4],
            [3,2]],

            [[16,24],
            [68, -2]]]
filter2 = [[[60,22],
            [32,18]],

            [[35,46],
            [7,23]],
            
            [[78,81],
            [20, 42]]]

input_array = [[[16,24, 32],
            [47, 18,26],
            [68, 12, 9]],

            [[26, 57, 43],
            [24, 21, 12],
            [2, 11, 19]],
            
            [[18, 47, 21],
            [4, 6, 12],
            [81, 22, 13]]]
filters = np.array([filter1, filter2])
inputs = np.array(input_array)
# print(filters)

conv.set_filter(filters)

# print(conv.calculate_receptive_map_size())
print(conv.forward(inputs))

[[[ 2171.  2170.]
  [ 5954.  2064.]]

 [[13042. 13575.]
  [11023.  6425.]]]


In [152]:
class DenseLayer:
    def __init__(self, size, activation, weights=[], name="denseboi"):
#       size example : (3,2) = there are 3 nodes, in which each has 2 coefficient
        if not weights:
            self.weights = np.random.rand(size[0], size[1])*5
        else:
            self.weights = weights
        self.shape = size
        self.output = self.weights
        self.bias = np.random.rand(size[0],1)
        self.act = activation
        self.name = name
        
    def __repr__(self):
        return f"\nDense Layer : {self.name}\nOutput Shape : {self.shape[0]}\nNParams : {self.shape[0] * self.shape[1]}\n"
    
    def forward(self, inp):
        # input should be array of Y length, where (X,Y) is the shape of the layer
        inp = inp.flatten()
        out = np.dot(np.concatenate((self.weights, self.bias),axis=1), np.append(inp,1))
        self.outputba = out
        self.output = self.act(out)
        return self.output

def ReLU(arr):
    return [(x if x > 0 else 0) for x in arr]

In [153]:
inp = InputLayer()

convo = ConvolutionalLayer((3,3,3),(0,0),2,(3,2,2),(1,1))
filter1 = [[[0,-1],
            [1,0]],

            [[5,4],
            [3,2]],

            [[16,24],
            [68, -2]]]
filter2 = [[[60,22],
            [32,18]],

            [[35,46],
            [7,23]],
            
            [[78,81],
            [20, 42]]]
filters = np.array([filter1, filter2])
convo.set_filter(filters)

dense = DenseLayer((3,8), ReLU)
out = OutputLayer()

arrlay = [inp, convo, dense, out]


In [154]:
seq = SeqModel(arrlay)

input_array = [[[16,24, 32],
            [47, 18,26],
            [68, 12, 9]],

            [[26, 57, 43],
            [24, 21, 12],
            [2, 11, 19]],
            
            [[18, 47, 21],
            [4, 6, 12],
            [81, 22, 13]]]

seq.inp(np.array(input_array))
seq.forward_propagate()

[176180.58442439063, 109330.0331371755, 143723.61047691936]

In [155]:
seq.summary()



Convolutional layer : convoboi
Output_Shape : 2*2*2
NParams : 27


Dense Layer : denseboi
Output Shape : 3
NParams : 24




In [2]:
with open("bitcoin_price_1week_Test - Test.csv") as f:
    testdata = f.read()
with open("bitcoin_price_Training - Training.csv") as f:
    traindata = f.read()

In [95]:
import pandas as pd
import numpy as np

x = pd.read_csv("bitcoin_price_1week_Test - Test.csv")
# data = []
# for idx in x.index:
#     data.append([x.loc[idx][i] for i in x.columns])
x

,Date,Open,High,Low,Close,Volume,Market Cap
0,"Aug 07, 2017",3212.78,3397.68,3180.89,3378.94,"1,482,280,000","52,987,300,000"
1,"Aug 06, 2017",3257.61,3293.29,3155.60,3213.94,"1,105,030,000","53,720,900,000"
2,"Aug 05, 2017",2897.63,3290.01,2874.83,3252.91,"1,945,700,000","47,778,200,000"
3,"Aug 04, 2017",2806.93,2899.33,2743.72,2895.89,"1,002,120,000","46,276,200,000"
4,"Aug 03, 2017",2709.56,2813.31,2685.14,2804.73,"804,797,000","44,666,400,000"
5,"Aug 02, 2017",2727.13,2762.53,2668.59,2710.67,"1,094,950,000","44,950,800,000"
6,"Aug 01, 2017",2871.30,2921.35,2685.61,2718.26,"1,324,670,000","47,321,800,000"


In [42]:
data = [x[1:] for x in data]
for i in range(len(data)):
    data[i][4] = int(data[i][4].replace(',',''))    
    data[i][5] = int(data[i][5].replace(',',''))    

In [92]:
data

[[3212.78, 3397.68, 3180.89, 3378.94, 1482280000, 52987300000],
 [3257.61, 3293.29, 3155.6, 3213.94, 1105030000, 53720900000],
 [2897.63, 3290.01, 2874.83, 3252.91, 1945700000, 47778200000],
 [2806.93, 2899.33, 2743.72, 2895.89, 1002120000, 46276200000],
 [2709.56, 2813.31, 2685.14, 2804.73, 804797000, 44666400000],
 [2727.13, 2762.53, 2668.59, 2710.67, 1094950000, 44950800000],
 [2871.3, 2921.35, 2685.61, 2718.26, 1324670000, 47321800000]]

In [100]:
from LSTMLayer import LSTMLayer

lstmlay = LSTMLayer(input_shape=(7,6))
model = SeqModel()
model.add(lstmlay)
# lstmlay.h = np.random.rand(2,1)
# lstmlay.c = np.random.rand(2,1)
# LSTMLayer()

In [102]:
inputs = []
for tes in data:
#     print(tes)
    tes = np.array(tes)
    inputs.append(tes)
#     print(tes.shape)
#     print(lstmlay.forward(tes))
inputs = np.array(inputs)
# print(inputs)
# lstmlay.forward(inputs)
model.inp(inputs)
print(model.forward_propagate())
# lstmlay.forward(inputs.reshape(-1,1))
# lstmlay.forward(inputs.reshape(-1,1))


[[3.21278e+03 3.39768e+03 3.18089e+03 3.37894e+03 1.48228e+09 5.29873e+10]
 [3.25761e+03 3.29329e+03 3.15560e+03 3.21394e+03 1.10503e+09 5.37209e+10]
 [2.89763e+03 3.29001e+03 2.87483e+03 3.25291e+03 1.94570e+09 4.77782e+10]
 [2.80693e+03 2.89933e+03 2.74372e+03 2.89589e+03 1.00212e+09 4.62762e+10]
 [2.70956e+03 2.81331e+03 2.68514e+03 2.80473e+03 8.04797e+08 4.46664e+10]
 [2.72713e+03 2.76253e+03 2.66859e+03 2.71067e+03 1.09495e+09 4.49508e+10]
 [2.87130e+03 2.92135e+03 2.68561e+03 2.71826e+03 1.32467e+09 4.73218e+10]]
